In [ ]:
import pandas as pd
import sklearn
import numpy as np
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import confusion_matrix
from sklearn.model_selection import train_test_split,GridSearchCV
from sklearn.ensemble import VotingClassifier,RandomForestClassifier

In [ ]:
data=pd.read_csv('/kaggle/input/gender-classification-dataset/gender_classification_v7.csv')

In [ ]:
data.head()

In [ ]:
data=data.sample(frac=1)
data.head()

In [ ]:
labels=data.pop('gender')
data.head()

In [ ]:
targets=labels.unique()
encoded_targets={target:label for label,target in enumerate(targets)}
print(encoded_targets)

In [ ]:
decode_targets={encoded_targets[label]:label for label in encoded_targets.keys()}
print(decode_targets)

In [ ]:
labels=labels.map(encoded_targets)

In [ ]:
print(labels)

In [ ]:
col_names=list(data.columns)
print(col_names)

In [ ]:
numeric_col_names=[]
for col_name in col_names:
    if type(data[col_name][0])==np.float64:
        numeric_col_names.append(col_name)
print(numeric_col_names)

In [ ]:
scaler=MinMaxScaler(feature_range=(0,1))
for num_col in numeric_col_names:
    scaler.fit(np.array(data[num_col]).reshape(-1,1))
    data[num_col]=scaler.transform(np.array(data[num_col]).reshape(-1,1))
data.head()

In [ ]:
X=np.array(data)
print(X)

In [ ]:
print(type(labels))

In [ ]:
y=np.array(labels)

In [ ]:
X_train,X_test,y_train,y_test=train_test_split(X,y,test_size=0.2)

In [ ]:
print(X_train.shape)
print(X_test.shape)
print(y_train.shape)
print(y_test.shape)

In [ ]:
parameters={'n_estimators':list(range(90,200,20)),'max_depth':list(range(200,400,40))}

In [ ]:
model=RandomForestClassifier()

In [ ]:
clf=GridSearchCV(model,parameters,verbose=3,refit=True)

In [ ]:
clf.fit(X_train,y_train)

In [ ]:
clf=clf.best_estimator_
print(clf)

In [ ]:
vote_clf=VotingClassifier(
    estimators=[(f'r{i}',clf) for i in range(0,10)],
    voting='soft',
    verbose=True
)

In [ ]:
vote_clf.fit(X_train,y_train)

In [ ]:
preds=vote_clf.predict(X_test)

In [ ]:
cm=confusion_matrix(y_true=y_test,y_pred=preds)
acc=cm.trace()/cm.sum()
print(f'The test set accuracy is {acc*100}')